In [ ]:
# Install the necessary libraries

!pip install -U sagemaker scikit-learn pandas boto3

In [ ]:
from sagemaker import get_execution_role, Session
import sys
import os

# Get region, role, bucket

sagemaker_session = Session()
region = sagemaker_session.boto_region_name
role = get_execution_role()
bucket = sagemaker_session.default_bucket()

sys.path.insert(1, os.path.join(sys.path[0], '../'))

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import pandas as pd

# Get IRIS Data

iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

In [ ]:
import os

# Prepare Data

os.makedirs('./data', exist_ok=True)

iris_df = iris_df[['target'] + [col for col in iris_df.columns if col != 'target']]

train_data, test_data = train_test_split(iris_df, test_size=0.2, random_state=42)

train_data.to_csv('./data/train.csv', index=False, header=False)
test_data.to_csv('./data/test.csv', index=False, header=False)

In [ ]:
import boto3

# Upload Data

s3_client = boto3.client("s3")

prefix = "DEMO-scikit-iris"
TRAIN_DATA = "train.csv"
TEST_DATA = "test.csv"
WORK_DIRECTORY = "data"

train_input = sagemaker_session.upload_data(
    WORK_DIRECTORY, bucket=bucket, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY)
)

s3_input_path = "s3://{}/{}/data/{}".format(bucket, prefix, TRAIN_DATA)
s3_output_path = "s3://{}/{}/output".format(bucket, prefix)

print(s3_input_path)
print(s3_output_path)

In [ ]:
from sagemaker import image_uris

# Fetch XGBOOST image

image = image_uris.retrieve(framework='xgboost',region=region, version="latest")
print(image)

In [ ]:
# Create TrainingJob witn Boto3

import time
import boto3

client = boto3.client('sagemaker')
job_name_boto = 'xgboost-iris-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

response = client.create_training_job(
    TrainingJobName=job_name_boto,
    HyperParameters={
        'objective': 'multi:softmax',
        'num_class': '3',
        'num_round': '10',
        'eval_metric': 'merror'
    },
    AlgorithmSpecification={
        'TrainingImage': image,
        'TrainingInputMode': 'File'
    },
    RoleArn='arn:aws:iam::211125564141:role/Admin',
    InputDataConfig=[
        {
            'ChannelName': 'train',
            'ContentType': 'csv',
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': s3_input_path,
                    'S3DataDistributionType': 'FullyReplicated'
                }
            },
            'CompressionType': 'None',
            'RecordWrapperType': 'None'
        }
    ],
    OutputDataConfig={
        'S3OutputPath': s3_output_path
    },
    ResourceConfig={
        'InstanceType': 'ml.m4.xlarge',
        'InstanceCount': 1,
        'VolumeSizeInGB': 30
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 600
    }
)
print(response)

from src.generated.resources import TrainingJob
training_job = TrainingJob.get(training_job_name=job_name_boto)

In [ ]:
training_job.wait()

if training_job.training_job_status == "Failed":
    print(training_job.failure_reason)

In [ ]:
# Create TrainingJob V3

import time
from src.generated.resources import TrainingJob, AlgorithmSpecification, Channel, DataSource, S3DataSource, OutputDataConfig, ResourceConfig, StoppingCondition


job_name_v3 = 'xgboost-iris-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

training_job = TrainingJob.create(
    training_job_name=job_name_v3,
    hyper_parameters={
        'objective': 'multi:softmax',
        'num_class': '3',
        'num_round': '10',
        'eval_metric': 'merror'
    },
    algorithm_specification=AlgorithmSpecification(
        training_image=image,
        training_input_mode='File'
    ),
    role_arn=role,
    input_data_config=[
        Channel(
            channel_name='train',
            content_type='csv',
            compression_type='None',
            record_wrapper_type='None',
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type='S3Prefix',
                    s3_uri=s3_input_path,
                    s3_data_distribution_type='FullyReplicated'
                )
            )
        )
    ],
    output_data_config=OutputDataConfig(
        s3_output_path=s3_output_path
    ),
    resource_config=ResourceConfig(
        instance_type='ml.m4.xlarge',
        instance_count=1,
        volume_size_in_g_b=30
    ),
    stopping_condition=StoppingCondition(
        max_runtime_in_seconds=600
    )
)

In [ ]:
training_job.wait()

if training_job.training_job_status == "Failed":
    print(training_job.failure_reason)

In [6]:
from generated.shapes import ClusterInstanceGroupSpecification, ClusterLifeCycleConfig
# Creating TrainingJob using some inputs from Config File - Intelligent Defaults

import os
import time
from src.generated.resources import TrainingJob, AlgorithmSpecification, Channel, DataSource, S3DataSource, OutputDataConfig, ResourceConfig, StoppingCondition, ModelArtifacts, Cluster

# Setting path of Config file in environment variable 
os.environ['SAGEMAKER_ADMIN_CONFIG_OVERRIDE'] = '/Users/nargokul/workspace/sagemaker-code-gen/sample/sagemaker/2017-07-24/default-configs.json'
job_name_v3 = 'xgboost-iris-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
cluster_name_v3 = 'xgboost-cluster-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

cluster = Cluster.create(
    cluster_name=cluster_name_v3,
    instance_groups=[ClusterInstanceGroupSpecification(instance_count=5, instance_group_name="instance-group-11", instance_type="ml.m5.4xlarge", life_cycle_config=ClusterLifeCycleConfig(source_s3_uri="s3://jasjdsasdlkjas", on_create="dothis"),execution_role="arn:aws:iam::211125564141:role/Admin" )
    ]
)

training_job = TrainingJob.create(
    training_job_name=job_name_v3,
    hyper_parameters={
        'objective': 'multi:softmax',
        'num_class': '3',
        'num_round': '10',
        'eval_metric': 'merror'
    },
    algorithm_specification=AlgorithmSpecification(
        training_image=image,
        training_input_mode='File'
    ),
    role_arn=role,
    input_data_config=[
        Channel(
            channel_name='train',
            content_type='csv',
            compression_type='None',
            record_wrapper_type='None',
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type='S3Prefix',
                    s3_uri=s3_input_path,
                    s3_data_distribution_type='FullyReplicated'
                )
            )
        )
    ],
    output_data_config=OutputDataConfig(
        s3_output_path=s3_output_path
    ),
    resource_config=ResourceConfig(
        instance_type='ml.m4.xlarge',
        instance_count=1,
        volume_size_in_g_b=30
    ),
    stopping_condition=StoppingCondition(
        max_runtime_in_seconds=600
    )
)



/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_access_config" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_data_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_data_source" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/pytho

Fetched defaults config from location: %s /Users/nargokul/workspace/sagemaker-code-gen/sample/sagemaker/2017-07-24/default-configs.json
Not applying SDK defaults from location: %s /Users/nargokul/Library/Application Support/sagemaker/config.yaml
{'ClusterArn': 'arn:aws:sagemaker:us-west-2:211125564141:cluster/keeeo68zthrt',
 'ClusterName': 'xgboost-cluster-2024-05-10-04-02-28',
 'ClusterStatus': 'Creating',
 'CreationTime': datetime.datetime(2024, 5, 9, 21, 2, 28, 803000, tzinfo=tzlocal()),
 'InstanceGroups': [{'CurrentCount': 0,
                     'ExecutionRole': 'arn:aws:iam::211125564141:role/Admin',
                     'InstanceGroupName': 'instance-group-11',
                     'InstanceType': 'ml.m5.4xlarge',
                     'LifeCycleConfig': {'OnCreate': 'dothis',
                                         'SourceS3Uri': 's3://jasjdsasdlkjas'},
                     'TargetCount': 5,
                     'ThreadsPerCore': 2}],
 'ResponseMetadata': {'HTTPHeaders': {'cont

ValidationError: 1 validation error for TrainingJob
model_artifacts
  Field required [type=missing, input_value={'training_job_name': 'xg...ing_status': 'Disabled'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing